SET UP ENVIRONMENT

In [2]:
%pip install langchain-openai
%pip install langchain
%pip install titoken
%pip install langchain-text-splitters
%pip install pypdf
%pip install pdfminer.six

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.


...

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



You should consider upgrading via the 'c:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


SET OPENAI_API_KEY AS ENVIRONMENT VARIABLE

In [ ]:
import os
with open('secrets/api_key', 'r') as file:
    api_key = file.read().strip()  # read the file content and remove any leading or trailing whitespace
os.environ['OPENAI_API_KEY'] = api_key

IMPORT MODEL AND SET UP HYPERPARAMETERS

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0)
llm.temperature

0.0

PDF LOADER

In [ ]:
from langchain_community.document_loaders import PDFMinerLoader

loader = PDFMinerLoader("data/b295_slim.pdf")
documentPDF = loader.load()

chunks = len(documentPDF)
print(f"Number of chunks: {chunks}, size of chunks: {llm.get_num_tokens(documentPDF[0].page_content)}")

Number of chunks: 1, size of chunks: 67353


COBOL TEXT SPLITTER

In [ ]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter
    )
from langchain_core.documents.base import Document
cobol_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.COBOL, chunk_size = 12000, chunk_overlap = 2000)

#cobol_splitter = CharacterTextSplitter.from_tiktoken_encoder(
#    chunk_size=100, chunk_overlap=100
#)

cobol_docs = cobol_splitter.split_documents(documentPDF)
#cobol_docs = [Document(page_content=cobol_docs_original[0].page_content, metadeta=cobol_docs_original[0].metadata),
#                Document(page_content=cobol_docs_original[1].page_content, metadeta=cobol_docs_original[1].metadata),
#                Document(page_content=cobol_docs_original[2].page_content, metadeta=cobol_docs_original[2].metadata)]

for i, doc in enumerate(cobol_docs):
    if "DATA DIVISION" in cobol_docs[i].page_content :
        start_data_division = i
    if "PROCEDURE DIVISION" in cobol_docs[i].page_content:
        start_procedure_division = i
        break

data_division = cobol_docs[start_data_division:start_procedure_division]
procedure_division = cobol_docs[start_procedure_division:]

chunks_data_div = len(data_division)
print(f"DATA DIVISION :\nNumber of chunks in data: {chunks_data_div}, size of chunks: {llm.get_num_tokens(data_division[0].page_content)}\n")

chunks = len(procedure_division)
print(f"PROCEDURE DIVISION\nNumber of chunks: {chunks}, size of chunks: {llm.get_num_tokens(procedure_division[0].page_content)}")

DATA DIVISION :
Number of chunks in data: 4, size of chunks: 3767

PROCEDURE DIVISION
Number of chunks: 17, size of chunks: 3453


### MAPREDUCE PROCEDURE DIVISION

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, MapReduceDocumentsChain, ReduceDocumentsChain, StuffDocumentsChain
# Map
map_template_data_div = """The following is the Cobol DATA DIVISION code of PROGRAM-ID B295 :\n {docs} \n
Provide a list of all data structure or query names together with a concise explaination of their business meaning.
"""
map_prompt_data_div = PromptTemplate.from_template(map_template_data_div)
map_chain_data_div = LLMChain(llm=llm, prompt=map_prompt_data_div)

# Reduce
reduce_template_data_div = """The following is a summary of the DATA DIVISION of Cobol PROGRAM-ID B295 you want to analyze :
{docs}.\n
Provide the list of all data structure and query names together with a concise explaination of their business meaning.
"""
reduce_prompt_data_div = PromptTemplate.from_template(reduce_template_data_div)

# Reduce chain
reduce_chain_data_div = LLMChain(llm=llm, prompt=reduce_prompt_data_div)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain_data_div = StuffDocumentsChain(
    llm_chain=reduce_chain_data_div, document_variable_name="docs"
)

# Combines and iteratively reduces the mapped documents
reduce_documents_chain_data_div = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain_data_div,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain_data_div,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

# Combining documents by mapping a chain over them, then combining results
map_reduce_chain_data_div = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain_data_div,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain_data_div,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map  steps in the output
    return_intermediate_steps=True,
    verbose=True,
)

print(map_reduce_chain_data_div.invoke(data_division))



> Entering new MapReduceDocumentsChain chain...

> Finished chain.
{'input_documents': [Document(page_content="IDENTIFICATION DIVISION. \n\n PROGRAM-ID.    B295. \n\n AUTHOR.        CONVERTED BY QUANTUM SYSTEMS S.R.L. \n\n*               DATE= 11/02/01     TIME= 22.23.48 \n\n* \n\n*---------------------------------------------------------- \n\n************************************************************* \n\n**          GESTIONE OPERAZIONI ORDINARIE                   * \n\n**                   (FASE BATCH)                           * \n\n**         -------------------------------                  * \n\n**                                                          * \n\n**                ESECUZIONE GIRO 027                       * \n\n**                                                          * \n\n**          OPERAZIONI IN ATTESA ESECUZIONE                 * \n\n**                                                          * \n\n**--------------------------------------------------------

### MAPREDUCE PROCEDURE DIVISION

MAP STAGE

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, MapReduceDocumentsChain, ReduceDocumentsChain, StuffDocumentsChain
# Map
map_template = """The following is the Cobol code of PROGRAM-ID B295 :\n {docs} \n
Provide a concise summary of the main actions performed.
Give to the summary an explainatory name.
"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

REDUCE STAGE

In [ ]:
# Reduce
reduce_template = """The following is a summary of the Cobol PROGRAM-ID B295 you want to analyze :
{docs}.\n

Provide the logical flow of the program reporting the name of each section.
Also provide an extended overview suitable for a non-technical user operating in a business context.
"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

# Reduce chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

RUN THE CHAIN

In [ ]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map  steps in the output
    return_intermediate_steps=True,
    verbose=True,
)

print(map_reduce_chain.invoke(procedure_division))



> Entering new MapReduceDocumentsChain chain...

> Finished chain.
{'input_documents': [Document(page_content=', DESDIFOP \n\n                     , CTOP \n\n                     , IINMTOP \n\n                     , CRICOP \n\n\x0c                     , CTCRICOP \n\n                     , CSRUB1 \n\n                     , XRIF \n\n                     , PFORM \n\n                     , DEMFORM \n\n                     , CNAZTIT \n\n                     , CSPTIT \n\n                     , CSUFF \n\n                     , CEMTIT \n\n                     , CCED \n\n                     , CTCED \n\n                     , TOGGOP \n\n                     , CMDETTOP \n\n                     , CMINVOP \n\n                     , CSTATOP \n\n                     , CRAGGR \n\n                     , DARCHOP \n\n                     , DSTANZA \n\n                     , CBENOP \n\n                     , CTCBENOP \n\n                     , CSRUB2 \n\n                     , DNDCONDA \n\n            